<a href="https://colab.research.google.com/github/CUknot/NLP/blob/main/Lab2_2_neural_language_modeling_to_student_2024.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Neural Language Modeling

In this Exercise, we will be using Pytorch Lightning to implement our neural LM. Your job will be just to write the forward method of the model.



## setup

In [ ]:
# #download corpus
!wget --no-check-certificate https://github.com/ekapolc/nlp_2019/raw/master/HW4/BEST2010.zip
!unzip BEST2010.zip

--2025-01-18 08:43:50--  https://github.com/ekapolc/nlp_2019/raw/master/HW4/BEST2010.zip
Resolving github.com (github.com)... 20.205.243.166
Connecting to github.com (github.com)|20.205.243.166|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/ekapolc/nlp_2019/master/HW4/BEST2010.zip [following]
--2025-01-18 08:43:51--  https://raw.githubusercontent.com/ekapolc/nlp_2019/master/HW4/BEST2010.zip
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 7423530 (7.1M) [application/zip]
Saving to: ‘BEST2010.zip’

BEST2010.zip        100%[===================>]   7.08M  --.-KB/s    in 0.08s   

2025-01-18 08:43:52 (90.9 MB/s) - ‘BEST2010.zip’ saved [7423530/7423530]

Archive:  BEST2010.zip
   creating: BEST2010/

In [ ]:
!pip install lightning

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.4/40.4 kB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 815.2/815.2 kB 37.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 927.3/927.3 kB 55.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 819.3/819.3 kB 43.8 MB/s eta 0:00:00


## code

In [ ]:
total_word_count = 0
best2010 = []
with open('BEST2010/news.txt','r',encoding='utf-8') as f:
  for i,line in enumerate(f):
    line=line.strip()[:-1] #remove the trailing |
    total_word_count += len(line.split("|"))
    best2010.append(line)

train = best2010[:int(len(best2010)*0.7)]
test = best2010[int(len(best2010)*0.7):]
#Training data
train_word_count =0
for line in train:
    for word in line.split('|'):
        train_word_count+=1
print ('Total sentences in BEST2010 news training dataset :\t'+ str(len(train)))
print ('Total word counts in BEST2010 news training dataset :\t'+ str(train_word_count))

Total sentences in BEST2010 news training dataset :	21678
Total word counts in BEST2010 news training dataset :	1042797


Here we are going to use a library from huggingface called `tokenizers`. This will help us create a vocabulary and handle the encoding and decoding, i.e., convert text to its corresponding ID (which will be learned by the tokenizer).

In [ ]:
from tokenizers import Tokenizer
from tokenizers.models import WordLevel
from tokenizers.pre_tokenizers import CharDelimiterSplit
from tokenizers.trainers import WordLevelTrainer

#Basically, we just use the new tokenizer as our vocab building tool.
#In practice, you will have to use a compatible tokenizer like newmm to tokenize the corpus first then do this step
tokenizer = Tokenizer(WordLevel(unk_token="[UNK]"))
tokenizer.pre_tokenizer = CharDelimiterSplit(delimiter="|") #now the tokenizer will split "|" for us
trainer = WordLevelTrainer(min_frequency=3,  #we can set a frequency threshold for taking a word into our vocab. for this example, words with freq < 3 will be excluded from the vocab.
                           special_tokens=["[UNK]", "<s>", "</s>"]) #these are our special tokens: for unknown, begin-of-sentence, and end-of-sentence, respectively.
tokenizer.train_from_iterator(train, trainer=trainer)

In [ ]:
len(tokenizer.get_vocab()) #same as nltk

9062

In [ ]:
tokenizer.encode("กฎหมาย|กับ|การ|เบียดบัง|คน|จน|asdf").tokens #tokens we get after tokenizing this sentence. unknown words will be tokenized as [UNK]

['กฎหมาย', 'กับ', 'การ', 'เบียดบัง', 'คน', 'จน', '[UNK]']

In [ ]:
tokenizer.encode("กฎหมาย|กับ|การ|เบียดบัง|คน|จน|asdf").ids #this is what we will feed to the LM

[242, 28, 5, 8883, 22, 190, 0]

In [ ]:
import itertools
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
import torch.optim as optim
import lightning as L
from tqdm import tqdm

In [ ]:
L.seed_everything(42, workers=True)

INFO: Seed set to 42
INFO:lightning.fabric.utilities.seed:Seed set to 42


42

In [ ]:
class TextDataset(Dataset):
  def __init__(self, data, seq_len = 128):
    #  data is currently a list of sentences
    #  [sent1,
    #   sent2,
    #   ...,
    #  ]

    data = [d+'|</s>' for d in data] #append an </s> token to each sentence
    encodings = tokenizer.encode_batch(data) #encode (turn token into token_id) data
    token_ids = [enc.ids for enc in encodings] #get the token ids for each sentence
    flatten_token_ids = list(itertools.chain(*token_ids)) #turn a list of token_ids into one long token_ids
    ## now data looks like this [sent1_ids </s> sent2_ids </s> ...]
    encoded = torch.LongTensor(flatten_token_ids)

    #remove some left over tokens so that we can form batches of seq_len (128 in this case). Optionally, we can use padding tokens instead.
    left_over = len(encoded) % seq_len
    encoded = encoded[:len(encoded)-left_over]
    self.encoded = encoded.view(-1, seq_len) #reshape data so it becomes a 2-D matrix of shape (len(encoded)//128, 128), i.e. each row contains data of len==128
    ## now data looks like this
    ## [ [1,2,3, ... , 128] (this is just an example, not actual input_ids)
    ##   [1,2,3, ... , 128]
    ##   [1,2,3, ... , 128]
    ## ]

  def __getitem__(self, idx):
    return self.encoded[idx]

  def __len__(self):
    return len(self.encoded)

In [ ]:
train_batch_size = 64
test_batch_size = 128
train_dataset = TextDataset(train)
train_loader = DataLoader(train_dataset, batch_size = train_batch_size, shuffle = True) #DataLoader will take care of the random sampling and batching of data

test_dataset = TextDataset(test)
test_loader = DataLoader(test_dataset, batch_size = test_batch_size, shuffle = False)

## Model : Implement the forward function here

> เพิ่มข้อความทั้งย่อหน้าที่ยกมา



In [ ]:
class LSTM(L.LightningModule):
    def __init__(self, vocab_size, embedding_dim, hidden_dim, num_layers, dropout_rate, learning_rate, criterion):

        super().__init__()

        self.num_layers = num_layers
        self.hidden_dim = hidden_dim
        self.embedding_dim = embedding_dim

        self.embedding = nn.Embedding(vocab_size, embedding_dim) #this will turn the token ids into vectors
        self.lstm = nn.LSTM(embedding_dim, hidden_dim, num_layers=num_layers,
                    dropout=dropout_rate, batch_first=True)
        self.dropout = nn.Dropout(dropout_rate)
        self.fc = nn.Linear(hidden_dim, vocab_size) #turn the vectors back into token ids
        self.learning_rate = learning_rate
        self.criterion = criterion

    def forward(self, src):
        embeddings = self.embedding(src)  # Shape: [batch_size, seq_len, embedding_dim]

        # Pass embeddings through the LSTM
        lstm_out, _ = self.lstm(embeddings)  # lstm_out: [batch_size, seq_len, hidden_dim]

        # Apply dropout
        lstm_out = self.dropout(lstm_out)

        # Project the hidden states to vocab size
        output = self.fc(lstm_out)  # Shape: [batch_size, seq_len, vocab_size]

        return output

    def training_step(self, batch, batch_idx):

        src = batch[:, :-1]
        target = batch[:, 1:]
        prediction = self(src) # run the sequence through the model (the forward method)
        prediction = prediction.reshape(-1, vocab_size)
        target = target.reshape(-1)
        loss = self.criterion(prediction, target)
        self.log("train_loss", loss)
        return loss

    def test_step(self, batch, batch_idx):

        src = batch[:, :-1]  #[batch_size (64) , seq_len-1 (127)] except last words
        target = batch[:, 1:] #[batch_size (64) , seq_len-1 (127)] except first words
        with torch.no_grad(): #disable gradient calculation for faster inference
          prediction = self(src) #[batch_size (64), seq_len-1 (127) , vocab size (9000)]
        prediction = prediction.reshape(-1, vocab_size) #[batch_size*(seq_len-1) (64*127=8128) , vocab]
        target = target.reshape(-1) #[batch_size (64), seq_len-1 (127)] -> [batch_size*(seq_len-1) (8128)]
        loss = self.criterion(prediction, target)
        self.log("test_loss", loss)
        return loss

    def configure_optimizers(self):
        return optim.Adam(self.parameters(), lr=self.learning_rate)

In [ ]:
vocab_size = tokenizer.get_vocab_size()
embedding_dim = 200
hidden_dim = 512
num_layers = 3
dropout_rate = 0.2
lr = 1e-3

In [ ]:
criterion = nn.CrossEntropyLoss()
model = LSTM(vocab_size, embedding_dim, hidden_dim, num_layers, dropout_rate, lr, criterion)

In [ ]:
from lightning.pytorch.loggers import CSVLogger
csv_logger = CSVLogger("log")

### Training

In [ ]:
trainer = L.Trainer(
    max_epochs=20,
    logger=csv_logger,
    deterministic=True
)

INFO: GPU available: True (cuda), used: True
INFO:lightning.pytorch.utilities.rank_zero:GPU available: True (cuda), used: True
INFO: TPU available: False, using: 0 TPU cores
INFO:lightning.pytorch.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO: HPU available: False, using: 0 HPUs
INFO:lightning.pytorch.utilities.rank_zero:HPU available: False, using: 0 HPUs


In [ ]:
trainer.fit(model, train_dataloaders=train_loader) # takes about 8 mins

INFO: LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO:lightning.pytorch.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO: 
  | Name      | Type             | Params | Mode 
-------------------------------------------------------
0 | embedding | Embedding        | 1.8 M  | train
1 | lstm      | LSTM             | 5.7 M  | train
2 | dropout   | Dropout          | 0      | train
3 | fc        | Linear           | 4.6 M  | train
4 | criterion | CrossEntropyLoss | 0      | train
-------------------------------------------------------
12.1 M    Trainable params
0         Non-trainable params
12.1 M    Total params
48.504    Total estimated model params size (MB)
5         Modules in train mode
0         Modules in eval mode
INFO:lightning.pytorch.callbacks.model_summary:
  | Name      | Type             | Params | Mode 
-------------------------------------------------------
0 | embedding | Embedding        | 1.8 M  | train
1 | lstm      | LSTM             | 5.7 M  | train
2 |

Training: |          | 0/? [00:00<?, ?it/s]

INFO: `Trainer.fit` stopped: `max_epochs=20` reached.
INFO:lightning.pytorch.utilities.rank_zero:`Trainer.fit` stopped: `max_epochs=20` reached.


### Testing

In [ ]:
test_result = trainer.test(model, dataloaders=test_loader)

INFO: LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO:lightning.pytorch.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Testing: |          | 0/? [00:00<?, ?it/s]

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃        Test metric        ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│         test_loss         │     4.164413928985596     │
└───────────────────────────┴───────────────────────────┘

In [ ]:
import numpy as np

In [ ]:
print(f"Perplexity : {np.exp(test_result[0]['test_loss'])}")

Perplexity : 64.35495481535708


In [ ]:
model.eval() #disable dropout

LSTM(
  (embedding): Embedding(9062, 200)
  (lstm): LSTM(200, 512, num_layers=3, batch_first=True, dropout=0.2)
  (dropout): Dropout(p=0.2, inplace=False)
  (fc): Linear(in_features=512, out_features=9062, bias=True)
  (criterion): CrossEntropyLoss()
)

In [ ]:
unk_token_id = tokenizer.encode("[UNK]").ids
eos_token_id = tokenizer.encode("</s>").ids

In [ ]:
def generate_seq(context, max_new_token = 10):
  encoded = tokenizer.encode(context).ids
  with torch.no_grad():
      for i in range(max_new_token):
          src = torch.LongTensor([encoded]).to(model.device)
          prediction = model(src)
          probs = torch.softmax(prediction[:, -1] / 1, dim=-1)
          prediction = torch.multinomial(probs, num_samples=1).item()

          while prediction == unk_token_id:
              prediction = torch.multinomial(probs, num_samples=1).item()

          if prediction == eos_token_id:
              break

          encoded.append(prediction)

  return tokenizer.decode(encoded)

In [ ]:
context = "<s>|วัน|จันทร์"
generate_seq(context, 50)

'วัน จันทร์ ที่   26   มิถุนายน     ปลัด   แจก ประชาสัมพันธ์ ผู้ว่า   นายก เทศมนตรี กรุงเทพมหานคร   ตชด.   รมต.   ร่วม กัน ได้ รับ ผล กระทบ ใน วัน พรุ่ง ที่   1   มิถุนายน   ที่ ผ่าน มา   ต่อ เข้า ประชุม ด้วย โดย  '

In [ ]:
context = "<s>|นายก"
generate_seq(context, 50)

'นายก ฯ   หรือ ต่อ คน   มี   กกต.   เพราะ เป็น เรื่อง ประกอบ ว่า   ใคร   ถึง เวลา ไหน มาก กว่า   ยัง เงียบหาย หรือ เทียบเท่า กัน   โดย สำนึก ความ ไม่ กล้า ใน ที่สุด   หรือ ทุก ชุด ที่ ดี อยู่แล้ว   ฐาน ทำลาย จริยธรรม ทั้งหมด  '

## Questions: Answer the following in MyCourseville

1. What is the perplexity of the neural LM you trained?
2. Paste your favorite sentence generated with the LM.
- นายก ฯ   หรือ ต่อ คน   มี   กกต.   เพราะ เป็น เรื่อง ประกอบ ว่า   ใคร   ถึง เวลา ไหน มาก กว่า   ยัง เงียบหาย หรือ เทียบเท่า กัน   โดย สำนึก ความ ไม่ กล้า ใน ที่สุด   หรือ ทุก ชุด ที่ ดี อยู่แล้ว   ฐาน ทำลาย จริยธรรม ทั้งหมด